In [1]:
import os
base_url = os.environ["API_BASE"]
print(base_url)

http://127.0.0.1:5000/


In [2]:
latitude = 65.1
longitude = -146.6

In [3]:
import micropip
await micropip.install('pyodide-http')
import pyodide_http
pyodide_http.patch_all()

In [5]:
import pandas as pd
df = pd.read_csv(f"{base_url}degree_days/freezing_index/{latitude}/{longitude}?format=csv", header=3)
df

,model,year,dd
0,ERA-Interim,1980,4605
1,ERA-Interim,1981,3430
2,ERA-Interim,1982,5504
3,ERA-Interim,1983,4891
4,ERA-Interim,1984,5013
...,...,...,...
215,NCAR-CCSM4,2096,2918
216,NCAR-CCSM4,2097,2067
217,NCAR-CCSM4,2098,2459
218,NCAR-CCSM4,2099,2224


In [6]:
await micropip.install("ipywidgets")
import ipywidgets as widgets
from IPython.display import display

In [14]:
# function to compute design freezing index
def compute_and_display_design_index(start_year, end_year, n_coldest_years):
    
    df_slice = df[(df["year"] >= start_year) & (df["year"] < end_year)]
    design_index = round(df_slice.sort_values("dd", ascending=False).iloc[0:n_coldest_years]["dd"].mean())
    return design_index

In [13]:
# create interactive widgets for input
start_year_slider = widgets.IntSlider(
    value=2030, min=1980, max=2099, step=1, description="Start Year:"
)
end_year_slider = widgets.IntSlider(
    value=2060, min=1981, max=2100, step=1, description="End Year:"
)
n_coldest_years_slider = widgets.IntSlider(
    value=3, min=1, max=10, step=1, description="Number of Coldest Years:",
    style={"description_width": "initial"}
)

# create an output widget for dynamic displaying of result
output = widgets.Output()

# function to handle widget changes
def handle_slider_change(change):
    with output:
        output.clear_output()

        if (
            end_year_slider.value - start_year_slider.value
            < n_coldest_years_slider.value
        ):
            start_year_slider.value = (
                end_year_slider.value - n_coldest_years_slider.value
            )
            output.clear_output()
        result = compute_and_display_design_index(
            start_year_slider.value, end_year_slider.value, n_coldest_years_slider.value
        )
        print(f"Design Freezing Index: {result} °F•days")

# attach the change handler to the sliders
start_year_slider.observe(handle_slider_change, names="value")
end_year_slider.observe(handle_slider_change, names="value")
n_coldest_years_slider.observe(handle_slider_change, names="value")

handle_slider_change(None)

# display the widgets with the title and initial result
display(start_year_slider, end_year_slider, n_coldest_years_slider, output)
initial_result = compute_and_display_design_index(
    start_year_slider.value, end_year_slider.value, n_coldest_years_slider.value
)


IntSlider(value=2030, description='Start Year:', max=2099, min=1980)

IntSlider(value=2060, description='End Year:', max=2100, min=1981)

IntSlider(value=3, description='Number of Coldest Years:', max=10, min=1, style=SliderStyle(description_width=…

Output()